In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mnist=tf.keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_num=int(len(train_images)*0.8)
train_x=train_images[:train_num]
train_y=train_labels[:train_num]
valid_x=train_images[train_num:]
valid_y=train_labels[train_num:]
test_x=test_images
test_y=test_labels

train_x=train_x.reshape(-1,784)
train_y=tf.one_hot(train_y,depth=10)
valid_x=valid_x.reshape(-1,784)
valid_y=tf.one_hot(valid_y,depth=10)
test_x=test_x.reshape(-1,784)
test_y=tf.one_hot(test_y,depth=10)

train_x=tf.cast(train_x/255.0,tf.float32)
valid_x=tf.cast(valid_x/255.0,tf.float32)
test_x=tf.cast(test_x/255.0,tf.float32)

2023-03-11 01:03:08.006509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
w1=tf.Variable(tf.random.normal([784,64],mean=0.0,stddev=1.0,dtype=tf.float32))
b1=tf.Variable(tf.zeros([64]),dtype=tf.float32)
w2=tf.Variable(tf.random.normal([64,32],mean=0.0,stddev=1.0,dtype=tf.float32))
b2=tf.Variable(tf.zeros([32]),dtype=tf.float32)
w3=tf.Variable(tf.random.normal([32,10],mean=0.0,stddev=1.0,dtype=tf.float32))
b3=tf.Variable(tf.zeros([10]),dtype=tf.float32)
w=[w1,w2,w3]
b=[b1,b2,b3]

In [4]:
def plt_plot(image):
    plt.imshow(image.reshape(28,28),cmap='binary')
    plt.show()
def model(x,w,b):
    x=tf.nn.relu(tf.matmul(x,w[0])+b[0])
    x=tf.nn.relu(tf.matmul(x,w[1])+b[1])
    pred=tf.nn.softmax(tf.matmul(x,w[2])+b[2])
    return pred
def loss_function(x,y,w,b):
    pred=model(x,w,b)
    loss_=tf.keras.losses.categorical_crossentropy(y_true=y,y_pred=pred)
    return tf.reduce_mean(loss_)
def grad(x,y,w,b):
    with tf.GradientTape() as tape:
        return tape.gradient(loss_function(x,y,w,b),w+b)
def accuracy(x,y,w,b):
    pred=model(x,w,b)
    correct_prediction=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  

In [5]:
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

loss_list_train=[]
loss_list_valid=[]
acc_list_train=[]
acc_list_valid=[]

In [6]:
train_epochs=5
batch_size=50
total_step=int(train_num/batch_size)
for epoch in range(train_epochs):
    for step in range(total_step):
        xs=train_x[step*batch_size:(step+1)*batch_size]
        ys=train_y[step*batch_size:(step+1)*batch_size]
        grads=grad(xs,ys,w,b)
        optimizer.apply_gradients(zip(grads,w+b))

    loss_list_train.append(loss_function(train_x,train_y,w,b).numpy())
    loss_list_valid.append(loss_function(valid_x,valid_y,w,b).numpy())
    acc_list_train.append(accuracy(train_x,train_y,w,b).numpy())
    acc_list_valid.append(accuracy(valid_x,valid_y,w,b).numpy())
    print('epoch={:2d}, train loss={:.4f}, train accuracy={:.4f}, value loss={:.4f}, value accuracy=:{:.4f}'
    .format(epoch+1,loss_list_train[-1],acc_list_train[-1],loss_list_valid[-1],acc_list_valid[-1]))

epoch= 1, train loss=6.9348, train accuracy=0.5559, value loss=6.8983, value accuracy=:0.5573
epoch= 2, train loss=3.9643, train accuracy=0.7413, value loss=3.9019, value accuracy=:0.7452
epoch= 3, train loss=3.1103, train accuracy=0.7960, value loss=2.9847, value accuracy=:0.8045
epoch= 4, train loss=2.5030, train accuracy=0.8347, value loss=2.3385, value accuracy=:0.8461
epoch= 5, train loss=1.9964, train accuracy=0.8684, value loss=1.9212, value accuracy=:0.8728
